# Data Cleansing
These notebook will explore our raw data after the AWS Lambda data pipeline. The final output of that pipeline is a large amount of JSON data files dumped to S3 with data coming from Craigslist, Mapquest and WalkScore.com.

## Import Our Libraries

In [171]:
import boto3
import pandas as pd
import Config
from pprint import pprint
import json
import numpy as np
import re

## AWS Credentials

In [172]:
# Get Credentials
credentials = Config.read_credentials()
aws_secret = credentials['aws']['aws_secret_access_key']
aws_access_key = credentials['aws']['aws_access_key_id']

## Bring in the raw JSON data

In [173]:
# Get keys of files
s3 = boto3.client('s3',
                 aws_access_key_id=aws_access_key,
                 aws_secret_access_key=aws_secret)

bucket = 'lazyapartment'
objects = s3.list_objects_v2(Bucket='lazyapartment', Prefix='walkScoreEnhancedData/')

## Combine all the raw JSON to Pandas for exploration

In [174]:
# Put all of the keys into a list
keys = [obj['Key'] for obj in objects['Contents']][1:]

# Put all the raw data into a pandas dataframe
data = []
s3 = boto3.resource('s3')
for key in keys:
    bucket_object = s3.Object(bucket, key)
    contents = bucket_object.get()['Body'].read().decode('utf-8')
    json_data = json.loads(contents)
    data.append(json_data[0])

df = pd.DataFrame(data)

# Export for easy access later
df = df.set_index('id')
df = df.to_csv('housing.csv')

## Export the DF for easy access later

In [226]:
df = pd.read_csv('housing.csv')

## Clean the data

### Separate out latitude and longtidue, drop Geotag column

In [227]:
df['lat'] = df['geotag'].apply(lambda x: x[1:x.index(',')])
df['lon'] = df['geotag'].apply(lambda x: x[x.rindex(',')+1:-1])
df.drop(columns='geotag', axis=1, inplace=True)

### Clean up area
Unfortunately many of the apartment listings don't have an area in square feet included. For those that do have square footage they come in as strings, so here we remove "ft2" from the column and convert it to a numeric datatype. We can also make a feature for whether or not the posting includes the square footage.

In [228]:
df.head(20)
def cleanUpArea(row):
    if type(row) == str:
        row = int(row.replace('ft2', ''))
    else:
        row
    return row

df['area'] = df['area'].apply(lambda x: cleanUpArea(x))
df['includes_area'] = df['area'].apply(lambda x: 0 if np.isnan(x) else 1)

### Clean up Price
Similarly price is a string as it is prefixed with a '$'

In [229]:
df['price'] = df['price'].apply(lambda x: x.replace('$', '')).astype(int)

### Dates
Convert the datetime field to a datetime and extract features

In [230]:
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M')
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['dow'] = df['datetime'].dt.dayofweek
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour

### Missing Bedrooms
For now we will hackily parse for information about the bedrooms that don't include the information. Eventually this seems to be a not very difficult NLP problem to solve once enough data is acquired. I say not very difficult as a 2 bedroom apartment is often, in text, described with either "2BR", "2BD" or "2 Bedroom" which should be easy enough to solve with NLP.

In [231]:
def parseBedrooms(row):
    if np.isnan(row['bedrooms']):
        if any(re.findall(r'studio', row['name'], re.IGNORECASE)):
            return 0
        elif any(re.findall(r'1BD|1 BD|1BR|1 BR|1 bedroom|1bedroom|one bedroom', row['name'], re.IGNORECASE)):
            return 1
        elif any(re.findall(r'2BD|2 BD|2BR|2 BR|2 bedroom|2bedroom|two bedroom', row['name'], re.IGNORECASE)):
            return 2
        elif any(re.findall(r'3BD|3 BD|3BR|3 BR|3 bedroom|3bedroom|three bedroom', row['name'], re.IGNORECASE)):
            return 3
    else:
        return row['bedrooms']

df['bedrooms_filled'] = df.apply(parseBedrooms, axis=1)

In [232]:
missing_bedrooms = df[df['bedrooms_filled'].isnull()]
missing_bedrooms.to_csv('missing_bedrooms.csv')